In [1]:
import pandas as pd

In [2]:
# Read in excel spreadsheet of data
df = pd.read_excel("Height_0_20198281030_QTOF LIB Run2 08082014_MSMS Hits only.xlsx")

In [3]:
# Because of how the spreadsheet is organized, the fourth row contains the column names we want
cols = df.loc[3]

In [4]:
# Remove the rows we don't need
df = df.loc[4:]

In [5]:
# Rename the columns to their proper names
df.columns = cols

In [6]:
# Extract only the columns that we need
msp_info = df[['Average Rt(min)', 'Average Mz', 'Metabolite name', 'Adduct type', 
               'Formula', 'INCHIKEY', 'MS1 isotopic spectrum', 'MS/MS spectrum']]

In [7]:
# The rows are numbered incorrectly, so reset the index 
msp_info = msp_info.reset_index().rename_axis(None, axis=1).drop('index',axis=1)

In [8]:
# Take a series of MS/MS values, return a series of lines w/mass and intensity
def msms_lines(msms):
    lines = []
    # Separate into mass-intensity pairs
    ms_pairs = msms.strip().split(" ")
    # Split pairs into there mass and intensity and append to lines
    for pair in ms_pairs:
        lines.append(pair.replace(":"," ") + "\n")
    return lines

In [9]:
# Write the information in a .msp file
test_msp = open("test_msp.msp", "w")
lines = []
for i in range(len(msp_info)):
    lines = [
        "Name: " + msp_info.loc[i]['Metabolite name'] + "\n",
        "InChIKey: " + msp_info.loc[i]['INCHIKEY'] + "\n",
        "Precursor Type: " + msp_info.loc[i]['Adduct type'] + "\n",
        "Precursor Mz: " + str(msp_info.loc[i]['Average Mz']) + "\n",
        "Retention Time: " + str(msp_info.loc[i]['Average Rt(min)']) + "\n",
        "Formula: " + msp_info.loc[i]['Formula'] + "\n",
    ]
    ms_lines = msms_lines(msp_info.loc[i]['MS/MS spectrum'])
    lines.append("Num Peaks: " + str(len(ms_lines)) + "\n")
    test_msp.writelines(lines)
    ms_lines.append("\n\n\n")
    test_msp.writelines(ms_lines)
test_msp.close()